In [6]:
import pandas as pd

In [7]:
# read the dataset 
NEISS_df=pd.read_csv('NEISS2014.csv')
#NEISS_df.info()
NEISS_df.head(3)

#TODO handling NA columns
#not sure about this method with respect to different columns and whether we should be dropping data or just replacing it
#NEISS_df.dropna()
#NEISS_df.fillna(value)


,CPSC Case #,trmt_date,psu,weight,stratum,age,sex,race,race_other,diag,diag_other,body_part,disposition,location,fmv,prod1,prod2,narrative
0,141200216,10/23/14,63,99.704,M,21,Female,None listed,NaN,62,NaN,75,1,1,0,679,1807.0,21YOF LAYING ON THE COUCH AND ROLLED OFF TO TH...
1,140117851,1/4/14,63,81.576,M,62,Female,White,NaN,57,NaN,79,1,1,0,1807,115.0,62YOF TRIPPED OVER THE CORD FOR THE VACUUM CLE...
2,150144993,8/17/14,63,99.704,M,21,Female,None listed,NaN,57,NaN,83,1,0,0,1333,NaN,21YOF FELL OFF A SKATEBOARD AND FRACTURED FOOT


In [8]:
bodyparts=pd.read_csv('BodyParts.csv')
bodyparts.head(3)

,BodyPart,Code
0,"Arm, lower",33
1,"Arm, upper",80
2,Ankle,37


In [13]:
diagnosis_codes=pd.read_csv('DiagnosisCodes.csv')
diagnosis_codes.head(3)

,Diagnosis,Code
0,Amputation,50
1,Anoxia,65
2,Aspirated foreign object,42


# Question 1

In [14]:
#Question 1
body_part_count=NEISS_df['body_part'].value_counts()

# alterntively could have done this as well rather than using value_counts
#NEISS_df.groupby('body_part').size().sort_values(ascending=False).head(3)

# Top 3 body parts  most frequently represented in the dataset
top3_mostfrequent_body_part=body_part_count.head(3)
top3_mostfrequent_body_part_df = pd.DataFrame({'Code':top3_mostfrequent_body_part.index, 'Frequency':top3_mostfrequent_body_part.values})

pd.merge(top3_mostfrequent_body_part_df, bodyparts, how='inner', on='Code')


,Code,Frequency,BodyPart
0,75,9891,Head
1,76,5786,Face
2,92,5783,Finger


In [15]:
# Top 3 body parts least frequently represented in the dataset
top3_leastfrequent_body_part=body_part_count.tail(3)
top3_leastfrequent_body_part = pd.DataFrame({'Code':top3_leastfrequent_body_part.index, 'Frequency':top3_leastfrequent_body_part.values})

pd.merge(top3_leastfrequent_body_part, bodyparts, how='inner', on='Code')

,Code,Frequency,BodyPart
0,87,390,Not Recorded
1,38,286,Pubic region
2,84,4,25-50% of body


# Question 2

In [16]:
#Question 2

#from the pdf file product code for skateboard injuries is 1333

skateboard_injuries_df = NEISS_df[(NEISS_df['prod1'] == 1333) | ((NEISS_df['prod2'] == 1333))]
skateboard_injuries_df_2 = skateboard_injuries_df[['age', 'sex']]

#Total number of skateboard injuries
skateboard_injuries_total = len(skateboard_injuries_df.index)

print("Total number of skateboard injuries: %d"%(skateboard_injuries_total))

Total number of skateboard injuries: 495


In [21]:
#male and female injuries involving skateboard
skateboard_injuries_male = len(skateboard_injuries_df_2[skateboard_injuries_df_2['sex'] == 'Male'].index)
skateboard_injuries_female = len(skateboard_injuries_df_2[skateboard_injuries_df_2['sex'] == 'Female'].index)

print("approx male injuries: %0.2f"%(float(skateboard_injuries_male*100)/skateboard_injuries_total))
print("approx female injuries: %0.2f"%(float(skateboard_injuries_female*100)/skateboard_injuries_total))


approx male injuries: 82.02
approx female injuries: 17.98


In [26]:
#Average age of someone injured in skateboard injury
skateboard_injuries_df_2['age'].mean()

#FIXME:Haven't yet converted the age given in months into years. So, the above answer might be skewed
#Basic check suggests that the skateboard injuries does not include age>=200 i.e (age given in months)
#len(skateboard_injuries_df_2[skateboard_injuries_df_2['age']>=200])

17.89090909090909

# Question 3

In [28]:
#Question 3

#select all the rows that correspond to hospitalization
#assume codes 2, 4, 5 are meant for hospitalization, filtering out rows that correspond to hospitalization
hospitialization_df = NEISS_df[NEISS_df['disposition'].isin([2,4,5])]

hospitialization_diagnosis_frequency = hospitialization_df['diag'].value_counts()
hospitialization_diagnosis_df = pd.DataFrame({'Code':hospitialization_diagnosis_frequency.index, 'Frequency':hospitialization_diagnosis_frequency.values})

# Diagnosis that had the highest hospitalization rate
pd.merge(hospitialization_diagnosis_df, diagnosis_codes, how='inner', on='Code').head(1)

,Code,Frequency,Diagnosis
0,57,2166,Fracture


In [31]:
#What diagnosis most often concluded with the individual leaving without being seen?

#assume code 6 is meant for leaving without being seen, filtering out rows that correspond to that

left_without_seen_df = NEISS_df[NEISS_df['disposition'] == 6]

left_without_seen_df_frequency = left_without_seen_df['diag'].value_counts()
left_without_seen_diagnosis_df = pd.DataFrame({'Code':left_without_seen_df_frequency.index, 'Frequency':left_without_seen_df_frequency.values})
pd.merge(left_without_seen_diagnosis_df, diagnosis_codes, how='inner', on='Code').head(1)


,Code,Frequency,Diagnosis
0,71,257,Other/Not Stated


In [ ]:
### Observations
# In case for dispostion leaving without being seen the above numbers are approximate and need more filtering, because:
# 1. Disposition code for leaving without being seen and  Left against medical advice are the same, 
# so there is a good chance that the above number has cases from `Left against medical advice`
# 2. Diagnosis `other/not stated ` is a broad diagnosis. We can further use the `diag_other` column to get more detailed analysis